# Blue Team KubeHound Workflow

A step by step example workflow of how to use KubeHound for an incident response scenario.

## Initial Setup

Connect to the kubegraph server by running the cell below

In [ ]:
%%graph_notebook_config
{
  "host": "host.docker.internal",
  "port": 8182,
  "ssl": false,
  "gremlin": {
    "traversal_source": "g",
    "username": "",
    "password": "",
    "message_serializer": "graphsonv3"
  }
}

Now set the appearance customizations for the notebook. You can see a guide on possible options [here](https://github.com/aws/graph-notebook/blob/623d43827f798c33125219e8f45ad1b6e5b29513/src/graph_notebook/notebooks/01-Neptune-Database/02-Visualization/Grouping-and-Appearance-Customization-Gremlin.ipynb#L680)

In [ ]:
%%graph_notebook_vis_options
{
  "edges": {
    "smooth": {
      "enabled": true,
      "type": "dynamic"
    },
    "arrows": {
      "to": {
        "enabled": true,
        "type": "arrow"
      }
    }
  }
}

## Workflow

### Compromised Credentials

Let us consider a scenario where a user's credentials have been compromised. We can use KubeHound to identify the resources that the user has access to, whether any lead to critical assets and what attacks might have been leveraged.

First let's see whether there are any critical paths accessible. Because Kubernetes delegates the management of users' group memberships to third party components (e.g identity providers), we need to check paths from both the user and any groups they are a member of.

In [ ]:
%%gremlin
kh.identities().
    or(
        has("type", "Group").has("name", within("dept-sales", "k8s-users")),
        has("type", "User").has("name", "bits.barkley@datadoghq.com")).
    hasCriticalPath().
    values("name")

Now let's examine the possible attack paths that could be taken by an attacker who has access to the compromised credebntials.

In [ ]:

%%gremlin -d class -g critical -le 50 -p inv,oute
kh.users("bits.barkley@datadoghq.com").
    criticalPaths().
    by(elementMap()).
    limit(100)


DONE Take a compromised credential and see the impact
Take a compromised container and see the impact

SHARED - do it woth the container case!
Examine the attack paths and look for potential detection sources
Eliminate those attacks to narrow down the scope
USe KH to focus the investigation on most likely attack paths and comrpomised hosts!